In [1]:
import numpy as np
import torch

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [3]:
torch.zeros((3,1)).T.cuda() @ torch.ones((3,1)).cuda()

tensor([[0.]], device='cuda:0')

In [4]:
import os
from torchvision.io import read_image
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [6]:
data_dir = "indoor_forward_9_snapdragon_with_gt/"
colnames=["stamp","name","none"]
my_data = pd.read_csv(data_dir + "left_images.txt", delimiter=' ', skiprows=1, names=colnames)
my_data = my_data.iloc[: , :-1]
my_data, len(my_data)

(             stamp                  name
 0     1.540823e+09     img/image_0_0.png
 1     1.540823e+09     img/image_0_1.png
 2     1.540823e+09     img/image_0_2.png
 3     1.540823e+09     img/image_0_3.png
 4     1.540823e+09     img/image_0_4.png
 ...            ...                   ...
 2063  1.540823e+09  img/image_0_2063.png
 2064  1.540823e+09  img/image_0_2064.png
 2065  1.540823e+09  img/image_0_2065.png
 2066  1.540823e+09  img/image_0_2066.png
 2067  1.540823e+09  img/image_0_2067.png
 
 [2068 rows x 2 columns],
 2068)

In [7]:
np.linalg.det(np.eye(3))

1.0

In [11]:
import transforms3d

In [18]:
alpha, beta, gamma = 0.123, -1.234, 2.345
origin, xaxis, yaxis, zaxis = [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]
R = transforms3d.euler.euler2mat(alpha, beta, gamma)
np.linalg.det(R) # this could be enforced in loss function ? : )

0.9999999999999999

In [42]:
# R = transforms3d.euler.euler2mat(alpha, beta, gamma)
R = np.random.rand(3,3)
R = torch.tensor(R, requires_grad=True)
Dloss = torch.norm(1 - R.det(),2)
Dloss.backward()

Dloss.item()

1.0007935808296207

In [47]:
np.linalg.norm([0.00712511541035951,-0.00599798947503901,0.890495788939425,-0.45489614939434]) # quaternion sample

1.0

In [50]:
# Td = T0**(-1) T1 could only take deltas of x,y and include those into loss function seperately
# the altitude & orientation could be mapped directly from the images, even from single instance. : )
# so the total loss function would be:
# loss = norm(1 - norm(q)) + norm(gt_q - q) + norm(gt_dx_dy - dx_dy) + norm(gt_z - z)

# could use padding to input variable length events ?

In [ ]:
data_dir = "indoor_forward_9_davis_with_gt/"
colnames=["stamp","name","none"]
my_data = pd.read_csv(data_dir + "events.txt", delimiter=' ', skiprows=1, names=colnames)
my_data = my_data.iloc[: , :-1]
my_data, len(my_data)